# Spain Electricity Shortfall Challenge


## Regression Project Student Solution

© Explore Data Science Academy

---

### Project Overview

The government of Spain is considering an expansion of it's renewable energy resource infrastructure investments. As such, they require information on the trends and patterns of the countries renewable sources and fossil fuel energy generation. Your company has been awarded the contract to:

- 1. analyse the supplied data;
- 2. identify potential errors in the data and clean the existing data set;
- 3. determine if additional features can be added to enrich the data set;
- 4. build a model that is capable of forecasting the three hourly demand shortfalls;
- 5. evaluate the accuracy of the best machine learning model;
- 6. determine what features were most important in the model’s prediction decision, and
- 7. explain the inner working of the model to a non-technical audience.

Formally the problem statement was given to you, the senior data scientist, by your manager via email reads as follow:

> In this project you are tasked to model the shortfall between the energy generated by means of fossil fuels and various renewable sources - for the country of Spain. The daily shortfall, which will be referred to as the target variable, will be modelled as a function of various city-specific weather features such as `pressure`, `wind speed`, `humidity`, etc. As with all data science projects, the provided features are rarely adequate predictors of the target variable. As such, you are required to perform feature engineering to ensure that you will be able to accurately model Spain's three hourly shortfalls.
 
On top of this, she has provided you with a starter notebook containing vague explanations of what the main outcomes are. 

<a id="cont"></a>

## Table of Contents

<a href=#zero>I. Problem Statement</a>

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Model Explanations</a>

 <a id="zero"></a>
## I. Problem Statement

To prevent the drastic effect of climate change and ensure sustainability of the global ecosystem, the world is gradually adopting the use of renewable energy. Asides ensuring a steady supply of electricity for a productive livelihood, renewable energy sources has also led to the emergence of new energy markets, enterprises, and job opportunities.

Renewable energy sources accounted for 43% of all electricity generated in Spain in the year 2020. As a result, the government of Spain is considering an expansion of its renewable energy resource infrastructure investments. To do so, they need information on the country's renewable resource and fossil fuel energy generating trends and patterns.

Our team of data scientists have been tasked with creating a model that would help predict the three-hourly load shortfall between the energy generated by means of fossil fuels and various renewable sources in Spain. This information will aid the government in determining how much infrastructure spending should be increased.

[Load Shortfall Image](https://dailytimes.com.pk/assets/uploads/2022/04/29/5ef6cf3f8fe3c.jpg)

![Loadshedding](https://github.com/JayHansea/TEAM-NM2/blob/65985167bb4b2ce180e3217d9b1b5356c9047a4d/Electricity%20Shortfall%20Image.jpg?raw=true)

[Image Source](https://dailytimes.com.pk/927865/pakistanis-suffer-worst-loadshedding-as-electricity-shortfall-reaches-9000mw/)



### II. OBJECTIVES

* Explore and visualize the dataset.
* Clean and engineer the dataset.
* Build several models that predicts the 3 hourly load shortfall.
* Assess the accuracy of the models.
* Choose the best model to make predictions.


### III. FEATURES DESCRIPTION
* **Time**: The date and time of the day when each feature value was recorded
* **Wind_speed**: This is a measure of the wind speed recorded in each city
* **Wind_deg**: This is a measure of the direction of the wind in each city
* **Pressure**: It is the atmospheric pressure measured in each city
* **Rain_1h/Rain_3h**: This is the amount of rain in each city as recorded in hourly or 3 hourly intervals
* **Snow**: The amount of snowfall in each city
* **Cloud_all**: This is a measure of the percentage of cloud coverage in each city


<a href=#cont>Back to Table of Contents</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>



---
    
| ⚡ Description: Importing Packages ⚡ |
| :--------------------------- |
| First we import, and briefly describe the libraries that will be used throughout our analysis and modelling. |

---

In [ ]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np   
import pandas as pd   

# Libraries for data preparation and model building
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import *
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


from tqdm import tqdm
import math
import random

#Visualization Packages
from scipy import stats
from scipy.stats import norm, skew 
import seaborn as sns 
from matplotlib import pyplot as plt
%matplotlib inline

# Suppresing warnings
import warnings
warnings.filterwarnings("ignore")

<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Importing & Previewing Data ⚡ |
| :--------------------------- |
|Here we imported the given datasets (train and test dataset) as sourced from [Kaggle](https://www.kaggle.com/competitions/spain-electricity-shortfall-challenge-2022/data). It is important to note that this dataset was first pushed to our Github repository from where it was then loaded to this notebook.

---

There are two data set provided in this project:
 - **df_train:** This is the data set that was used for the Exploratory Data Analysis (EDA) and model building and evaluatioin.
 
 - **df_test:** This data set was used for Kaggle submission. Data preprocessing and engineering were carried out on it as needed based on the works done on the train dataset and the dataset was fed into the trained model to make predictions for the Kaggle competition. The works on the dataset is covered in the kaggle submission notebook.

In [ ]:
# load the train data
df_train = pd.read_csv('Data/df_train.csv')
df_train.head()

In [ ]:
# Load the test data
df_test = pd.read_csv('Data/df_test.csv')
df_test.tail()

<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Exploratory data analysis ⚡ |
| :--------------------------- |
| In this section, we performed an in-depth analysis of all the variables in the DataFrame. |

---


### Checking the "shape" of the datasets

In [ ]:
df_train.shape

In [ ]:
df_test.shape

First, we checked the shape of both datasets, and observed that the training dataset has 49 columns while the test dataset has only 48 coulmns. The missing column from the test set is the target variable the (load_shortfall_3hr) that our model is to predict.

### Features Preview

It is imperative to have an overview of all features in the dataset

In [ ]:
features = pd.DataFrame(df_train.columns, columns=['Features'])

features

There unnamed column can be dropped as it does not represent any feature

In [ ]:
# Dropping the unnamed feature
df_train = df_train.drop(['Unnamed: 0'], axis = 1)

In [ ]:
df_train.head(2)

<br>

### Summary Statistics

The describe() function was used to generate descriptive statistics that summarizes the central tendency, dispersion 
and shape of the dataset, excluding null values.

In [ ]:
df_train.describe()

<br>

### Splitting the dataset by cities

In the following cells, we took a deeper dive into the train dataset by splitting it into the different cities. This was done to allow us assess the impact of city specific features in relation to the output variable.

In the following cells, a deeper look was taken at the train dataset by splitting it into different cities. This was done to assess the impact of city specific features in relation to the output variable - `load_shortfall_3h`.

**Checking the number of weather conditions recorded in each city and what those weather conditions are.**

In [ ]:
# Number of weather conditions recorded for each city

num_of_features = {}

for col in df_train.columns[1:-1]: # Excluding time and load_shortfall_3h columns
    city = col.split('_')[0]
    if city in num_of_features.keys():
        num_of_features[city] += 1
    else:
        num_of_features[city] = 1

# Viewing the number of weather conditins recorded for various cities in the dataset       
num_of_features

**Weather condition recorded for each city**

In [ ]:
# Weather conditions recorded for each city

city_weather_records = {}

for col in df_train.columns[1:-1]:  # Excluding time and load_shortfall_3h columns
    city = col.split('_')[0]
    if city in city_weather_records.keys():
        city_weather_records[city].append(col)
    else:
        city_weather_records[city] = [col]

# Viewing the number of weather conditins recorded for various cities in the dataset       
city_weather_records

<br>

### Null values
Identifying the column(s) with null entries is important so it doesnt affect the performance of the trained model. Using the isnull() function shows the number of null values present in the dataset. The function shows that only the `Valencia_pressure` feature has null values.

In [ ]:
# Checking for null values in the columns

df_train.isnull().sum()

In [ ]:
# Confirming the existence of null values in the column

df_train[df_train['Valencia_pressure'].isnull()]['Valencia_pressure']

In [ ]:
plt.figure(figsize=(5, 3))

sns.boxplot(df_train['Valencia_pressure'])

<br>

From the boxplot above, the `valencia_pressure` feature has values between 1010 and less than 1020

<br>

**Mean, Median and Mode of `Valencia_pressure`**

In [ ]:
# Checking the Summary statistics of Valencia_pressure

print('Mean:', df_train['Valencia_pressure'].mean(), '\n')

print('Median:', df_train['Valencia_pressure'].median(), '\n')

print('Mode:', df_train['Valencia_pressure'].mode())

### Choosing the measure to replace null values

As observed all three measures of central tendency have very similar values, as such there wouldn't be too much of a difference which measure we choose to go with. However, for the purpose of this model, we shalll replace the null values in "Valencia pressure" with the mean, since it is a better measure for the datatype.

In [ ]:
#Replacing the null values with the mean

df_clean_train = df_train
df_clean_train['Valencia_pressure'] = df_clean_train['Valencia_pressure'].fillna(df_clean_train['Valencia_pressure'].mean())

In [ ]:
# Confirming that the null values have been replaced

df_clean_train.isnull().sum()

In [ ]:
df_clean_train.shape

<br>

#### DATA TYPES

Machine learning models only work with numeric data, which means the data types for the models must be floats or integers in order to get the best predictions out of the models built from the dataset. Also, having the right data types is needed for visualizing the dataset during the Exploratory Data Analysis (EDA).The code below reveals the data types of the data contained in data set. Note that the `time` data, the `Valencia_wind_deg` data as well as the `Seville_pressure` are all object data (also known as strings). These have to be converted to floats or integers for them to be usable both for EDA and model training. Similar to the null values, they can either be dropped or converted.The former is not recommended as everytime a data is dropped, potentially valuable information that may be very useful for model building are lost. A more beneficial approach will be to process this data by Transforming it to numeric form or encoding it to a form that the model can utilize. 3 non-numeric objects are observed from the data set. they are `time`, `Valencia_wind_deg` and `Seville_pressure`


In [ ]:
df_clean_train.dtypes

<br>

### Modifying the time feature

The `time` feature is mapped into a datetime format which is the appropriate data type for date and time features and other features can be created from the date time feature that can be used in the model training.

In [ ]:
df_clean_train['time'] = pd.to_datetime(df_clean_train['time'])
df_clean_train.time

In [ ]:
df_clean_train.shape

<br>

### Visualizing the Load shortfall as a function of time

This datetime feature can now be used to create some time-dependent plots

In [ ]:
# Visualizing the load_shortfall as a function of time

sns.set(style="whitegrid") # Style for the plot
plt.figure(figsize=(20, 7)) # Setting the figure size

sns.lineplot(data=df_clean_train, x='time', y='load_shortfall_3h')

<br>

The plot above is showing very little information. Visualizing the load shortfall as a function of other date-time parameters may provide better insight to the pattern of the load short fall.

As such other date time parameters were extracted from the time feature

### Extracting other Datetime feature from the time feature

In [ ]:
column_list =[]

# second
df_clean_train['second'] = df_clean_train['time'].dt.second

# minute
df_clean_train['minute'] = df_clean_train['time'].dt.minute

# hour
df_clean_train['hour'] = df_clean_train['time'].dt.hour

# day
df_clean_train['Day'] = df_clean_train['time'].dt.day

# month
df_clean_train['Month'] = df_clean_train['time'].dt.month

# year
df_clean_train['Year'] = df_clean_train['time'].dt.year


# adding the new features to the dataset 
column_list = ['time', 'second', 'minute', 'hour','Day','Month','Year'] + list(df_clean_train.columns[1:-6])



df_clean_train = df_clean_train[column_list]


In [ ]:
df_clean_train.head(3)

<br>

**Checking the nature of values in the `second` and `minute` columns**

In [ ]:
# Checking the number of entries in the second and minute columns that are zeros

df_clean_train[(df_clean_train['second'] == 0) & (df_clean_train['minute'] == 0)][['second', 'minute']].count()


From the result above, the values in the `second` and `minute` columns are all zeros and as such will be dropped from the dataframe.

It is equally necessary to check the range of values in the remaining date and time columns, that is, `hour`, `day`, `month`, and `year`

<br>

**Dropping the `second` and `minute` columns from the dataframe**

In [ ]:
df_clean_train.drop(columns=['second', 'minute'], inplace=True)

<br>

**Number of unique values in all datetime columns**

In [ ]:
# Number of unique values in the hour column

df_clean_train['hour'].unique()

It's reasonable to see that the hour are in multiples of three as the shortfalls were recorded every three hours

In [ ]:
# Number of unique values in the Day column

df_clean_train['Day'].unique()

In [ ]:
# Number of unique values in the Month column

df_clean_train['Month'].unique()

In [ ]:
# Number of unique values in the Year column

df_clean_train['Year'].unique()

<br>

### Visualizing the Load shortfall with respect to `Hour`, `Day`, and `Month`

It is rather intuitive to visualize the `load_shortfall` in `hour`, `Day`, and `Month`.

To achieve this, the following were done:
 - The index of the dataframe used in plotting these visualizations was set to be the time.
 - The dataframe was resampled based on the time interval being considered

In [ ]:
# Visualizing the load_shortfall for every 3 hours intervals

print()

plt.figure(figsize=(25, 7))

sns.set(font_scale=1.5)
sns.set_style("darkgrid")

plt.plot(df_clean_train.set_index('time').resample('H').ffill()['load_shortfall_3h'])

plt.title("3-Hourly Load Shortfall")
plt.xlabel("Time (3-Hourly)")
plt.ylabel("Load Shortfall")
plt.show()

<br>

From the plot above, the shortfall can be seen to show some regular pattern with peaks at specific points and intervals. This visualization can be drilled down to get better intuition into this pattern.

In [ ]:
# Visualizing the load_shortfall for every day

print()

plt.figure(figsize=(20, 7)) # Setting the figure size

sns.set(font_scale=1.5)
sns.set_style("darkgrid")

#sns.lineplot(data=df_clean_train, x='Day', y='load_shortfall_3h')
plt.plot(df_clean_train.set_index('time').resample('D').ffill()['load_shortfall_3h'])

plt.title("Daily Load Shortfall")
plt.xlabel("Day of week")
plt.ylabel("Load Shortfall")
plt.show()

<br>

The shortfall shows inherent pattern across days of week and this will be further investigated by lookingat the pattern within months

In [ ]:
# Visualizing the load_shortfall for every month

print()

plt.figure(figsize=(15, 3)) # Setting the figure size

sns.set(font_scale=1)
sns.set_style("darkgrid")

plt.plot(df_clean_train.set_index('time').resample('W').ffill()['load_shortfall_3h'])

plt.title("Weekly Load Shortfall")
plt.xlabel("Week")
plt.ylabel("Load Shortfall")
plt.show()

<br>

<br>

In [ ]:
# Visualizing the load_shortfall for every month

print()

plt.figure(figsize=(15, 3)) # Setting the figure size

sns.set(font_scale=1)
sns.set_style("darkgrid")

plt.plot(df_clean_train.set_index('time').resample('M').ffill()['load_shortfall_3h'])

plt.title("Load Shortfall versus Month")
plt.xlabel("Month")
plt.ylabel("Load Shortfall")
plt.show()


In the visualization above, the shortfall can be seen to follow a trend across the three years interval of the dataset.


### Distribution of weather data across cities

To get a better intuiton into the nature of the data set and see how different weather records compare across different cities it is imperative to visualize different weather conditions across cities.

In [ ]:
# Weather conditions recorded for all the cities excluding wind degree and 3-hourly rainfall

weather_type_records = {}

for col in df_clean_train.columns[5:-1]:  # Excluding time and load_shortfall_3h columns
    weather_type = col.split('_')[1]
    #if (col.split('_')[-1] not in ['deg', '3h']):
    if (weather_type in weather_type_records.keys()):
        weather_type_records[weather_type].append(col)
    else:
        weather_type_records[weather_type] = [col]

# Viewing the number of weather conditins recorded for various cities in the dataset       
weather_type_records

<br>

**The Histograms below help in visualizing the nature of the dataset and how they compare among themselves.**

In [ ]:
# Histograms to show the distribution of weather conditions for different cities.

for weather in weather_type_records.keys():
    cities_weather = weather_type_records[weather]
    weather_name = f'{weather}_df'
    weather_name = df_clean_train[cities_weather]
    
    print()
    
    sns.set_context(font_scale=.2)
    weather_name.hist(layout = (4, 4), figsize=(10, 8))
    
    plt.tight_layout(pad=3.0)
    plt.show()


In [ ]:
# Histograms to show the distribution of weather conditions for different cities.

for weather in weather_type_records.keys():
    cities_weather = weather_type_records[weather]
    weather_name = f'{weather}_df'
    weather_name = df_clean_train[cities_weather]
    
    print()
    
    weather_name.plot(kind='density', subplots=True, layout=(4,4), sharex=False, figsize=(10,8))
    
    plt.tight_layout(pad=2.0)
    plt.show()


<br>

The histograms above shows the distribution of the various weather conditions for the different cities. The following can be observed:
 - The wind speed values are left skewed with majority of the values between 0 and 5. Some
 - The wind degree have a different distribution from the wind speed, with a distribution spread around 200
 - The one-hourly rain measure has has most of its value close to 0 and less than 1 while the 3 hourly rain measure are less than 0.0. This measures will be further explored to decide whether to drop them from the dataframe or not before proceeding to the model training stage.
 - The three cities with humidity measure have the distributions range less than 100
 - For the cloud measure, the majority of the distribution are very close to zero except for that of Bilbao
 - The distribution of the pressure for all cities are between 975 and 1025 with most of the data being above 1000 except for Barcelona which seems to have been recorded using a different unit.
 - The 3-hourly snow measure both have different range of values and this will be further explored.
 - All the weather_id of the cities have similar distribution and what this data represents should be explored.
 - Every of the temperature distribution are similar and the min and max features will be dropped, but that will be delayed till later.


<br>

**Line plots of weather data across cities|**

The trend of each weather data recorded across the cities will be explored here excluding wind degree and 3-hourly rainfall that have shown a large divergence from the rest of the data set in the distribution visualization done above.

In [ ]:
# Weather conditions recorded for all the cities excluding wind degree and 3-hourly rainfall

weather_type_records_2 = {}

for col in df_clean_train.columns[5:-1]:  # Excluding time and load_shortfall_3h columns
    weather_type = col.split('_')[1]
    if (col.split('_')[-1] not in ['deg']):
        if (weather_type in weather_type_records_2.keys()):
            weather_type_records_2[weather_type].append(col)
        else:
            weather_type_records_2[weather_type] = [col]

# Viewing the number of weather conditins recorded for various cities in the dataset       
#weather_type_records

**Creating the plots**

In [ ]:
# Line plots to show the distribution of weather conditions for different cities.

for weather in weather_type_records_2.keys():
    cities_weather = list(weather_type_records_2[weather])
    weather_name = f'{weather}'
    cities_weather.insert(0, 'time')
    weather_df = df_clean_train[cities_weather]
    
    print()
    
    sns.set_context(font_scale=1)
    weather_df.set_index('time').resample('M').ffill().plot(layout = (4, 4), figsize=(10, 3))
    plt.xlim(np.datetime64('2014-10-01'), np.datetime64('2018-04-01'))
    plt.title(weather_name)
    
    plt.tight_layout(pad=2.0)
    plt.show()


The plots above shows how each weather condition varies with time across each cities and as it can be seen, the distributions seem to exhibit some similarities except for some few.
 - Snow weather records seem to show little to no variation. This feature will be explored further to identify the nature of the data present in them and decide whether to leave or drop them from the dataframe.
 - Also, the temperature features all show very similar trends and as such there may be high multicollinearity between them. Moreover, the max and min values of these features can be dropped leaving only the actual features

<a id="four"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |
| In this section we cleaned the dataset, and created new features - as identified in the EDA phase. |

---

During the Exploratory Data Analysis, some data/feature engineering were done, such as investigating and fixing null values, datatypes of entries, and features with only zero entries. These were done to aid the exploratory data analysis that was done.

A this stage, further data/feature engineering will be carried out, more imprtantly for the purpose of the machine learning model training.

<br>

   From the plots of the distribution of weather data across cities that was created during Exploratory Data Analysis, it was pointed out that the one-hourly rain measure has has most of its value close to 0 and less than 1 while the 3 hourly rain measure are less than 0.0. This features call for further investigation and that is what will be done next.

### Exploring the rain features

**Checking for the number of zero entries in the three columns with rain data**

The code below can be used to obtain all rain features.

In [ ]:
# Getting a list of features that represents rain weather condition

rain_feature = []

for feature in df_clean_train.columns:
    if (len(feature.split('_')) > 1):
        if (feature.split('_')[1] == 'rain'):
            rain_feature.append(feature)
        

In [ ]:
# Obtaining the number of zero entries in all the rain features

df_clean_train[df_clean_train[rain_feature] != 0][rain_feature].count()

In [ ]:
df_clean_train.shape

<br>

Of all the values above, `Bilbao_rain_1h` has the highest number of no-zero values and this represents 29.6 % of its total entries. To be sure the zero entries are not due to approximation, the number of decimal places displayed by the dataframe was increased and a particular feature was checked. Below is what was obtained.

In [ ]:
pd.set_option('display.float_format', '{:.8f}'.format)

df_clean_train['Bilbao_rain_1h']

<br>

Rather than drop these features, they will be left in the dataframe as it is rather logical to have a few records for rainfal since there cannot be rainfall throughout the year. The high number of zero entries only implies that there are few instances of rainfall.

#### Number of Zero entries for each feature of the dataframe

In [ ]:
df_clean_train[df_clean_train[:] == 0][:].count()

<br>

Zero entries in the dataset for some features do not represent missing values but rather, shows the absence of that condition within that time frame of the measurement. Examples are the rain and snow features which contain a lot of zeros. This is logical as zero entries represent the absence of those weather conditions since weather conditions are expected to vary through the year.

### Data types

In [ ]:
df_clean_train.dtypes

There are two object data types in the dataframe. This two features will be investigated further.

In [ ]:
df_clean_train['Valencia_wind_deg']

<br>

**Number of unique entries in Valencia_wind_deg**

In [ ]:
df_clean_train['Valencia_wind_deg'].nunique()

The `Valencia_wind_deg` seems to be a categorical data represented by different levels. This levels, which are represented by integers, can be extracted for the purpose of encoding to train the machine learning model.

In [ ]:
# Extracting the digit from the strings of levels
df_clean_train['Valencia_wind_deg'] = df_clean_train['Valencia_wind_deg'].str.extract('(\d+)')

# Converting the extracted digits to integers
df_clean_train['Valencia_wind_deg'] = pd.to_numeric(df_clean_train['Valencia_wind_deg'])

# Checking the output
df_clean_train['Valencia_wind_deg']

<br>

`Seville_pressure` is another feature in the dataset that is still of type object, which is a string. The same process as above will be repeated on this feature to convert it to numeric.

In [ ]:
# Extracting the digit from the strings of levels
df_clean_train['Seville_pressure'] = df_clean_train['Seville_pressure'].str.extract('(\d+)')

# Converting the extracted digits to integers
df_clean_train['Seville_pressure'] = pd.to_numeric(df_clean_train['Seville_pressure'])

# Checking the output
df_clean_train['Seville_pressure']

<br>

**Viewing the features and their corresponding datatypes**

In [ ]:
df_clean_train.dtypes

In [ ]:
pd.DataFrame(df_clean_train.columns, columns=['Features'])

<br>

Before moving on to the model training, it will be important to test for multicollinearity, the correlation of each feature with the target variable and check for the significance of each feature.

### Evaluating correlation with the output variable

It is neccesary to evaluate the correlation of the different features in the train dataset with the target variable to ascertain which features have a very weak relationship with the target variable. This was done using the `corrwith` function of the Pandas library

In [ ]:
df_clean_train.corrwithb(df_clean_train["load_shortfall_3h"]).sort_values(ascending=False)

To test for multicollinearity, it will be more appealing to visualize the correlations between individual features and the target variable. This will be done using `Heatmap`

In [ ]:
correlation_matrix = df_clean_train.corr()

plt.figure(figsize=(15, 10))
heatmap = sns.heatmap(correlation_matrix, vmin=-1, vmax=1)
#heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

### Temperatures

From the Exploratory Data Analysis, all the temperature features tend to follow the same trend with time. It is imperative to investigate this features further to test for multicollinearity and their strength of their relationship with the target variable.


In [ ]:
# Getting all temperature features to compute the correlation with the target variable

temperature_list = []

for col in df_clean_train.columns:
    if (len(col.split('_')) > 1):
        if (col.split('_')[1] == 'temp'):
            temperature_list.append(col)


In [ ]:
temperature_list

## Variables Selection by Correlation and Significance

In [ ]:
column_titles = [col for col in x if col != 'load_shortfall_3h'] + ['load_shortfall_3h']

In [ ]:
from statsmodels.formula.api import ols

df_pval = df_modified[:len(df_train)].copy()

y_name = 'load_shortfall_3h'

x_names = [col for col in df_pval.columns if col != y_name]


formula_str = y_name + "~" + "+".join(x_names)
print('Formular:\n\t{}'.format(formula_str))

In [ ]:
model = ols(formula=formula_str, data=df_pval)

fitted = model.fit()

print(fitted.summary())

In [ ]:
corr_values = df_pval.corr()['load_shortfall_3h'].sort_values(ascending=False)

corr_values

In [ ]:
from scipy.stats import pearsonr

dict_corr_p = {}

column_titles_pr = [col for col in corr_values.index if col not in ['load_shortfall_3h', 'minute', 'second']]
for col in column_titles_pr:
    p_val = round(pearsonr(df_pval[col], df_pval['load_shortfall_3h'])[1], 6)
    dict_corr_p[col] = {'Correlation coefficient': corr_values[col],
                       'P_Value': p_val}
    
df_corr_p = pd.DataFrame(dict_corr_p).T
df_corr_p_sorted = df_corr_p.sort_values('P_Value')
df_corr_p_sorted[df_corr_p_sorted['P_Value']<0.1]

In [ ]:
x_names_sig = list(df_corr_p.index)

x_data = x[x_names_sig]

m_col = x_data.corr()

r, c = np.where(np.abs(m_col)>0.9)

off_diagonal = np.where(r != c)

multicol_df = m_col.iloc[r[off_diagonal], c[off_diagonal]]

<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, we created two models that enable us to accurately predict the thee hour load shortfall. |

---



In [ ]:
# split data
y = df_modified_train[['load_shortfall_3h']]
x = df_modified_train.drop('load_shortfall_3h',axis=1)

In [ ]:
x.head()

In [ ]:
y.head()

### Building One or More ML Models

First we created a base model, and two other models. In this case, our base model is a simple linear regression model. Other models created to improve predictive performance are the random forest and lasso regression models.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.34,random_state=50)

#### i. Linear Regression Model

In [ ]:
# Declare the model object
lm = LinearRegression()

# fit linear model
lm.fit(x_train, y_train)

prediction_lm = lm.predict(x_test)


#### ii. Random Forest Regression Model
Random forests or random decision forests is an ensemble learning method. In the following cells, the random forest regression model will be loaded, trained and used to predict the load shortfall from the given data set.

In [ ]:
# Fitting Random Forest Regression to the dataset

# create regressor object
Rfr = RandomForestRegressor(n_estimators = 100, max_depth=15, random_state=18)

Rfr.fit(x_train, np.ravel(y_train))

predictions_Rfr_test = Rfr.predict(x_test)

#### iii. LASSO Regression Model

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso()

lasso.fit(x_train, y_train)

predictions_lasso_test = lasso.predict(x_test)


<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section we compared the relative performance of the various trained ML models on a holdout dataset and indicated which is the best and why. |

---

### The root mean square value (RMSE)

The root mean squared error (RMSE) is a frequently used measure of the differences between values (sample or population values) predicted by a model or an estimator and the actual values observed. It is a very useful tool in telling how well the model predicted the values using the test dataset. Below is a function that calculates and returns the
average RMSE of the model.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
# Compare model performance
# LINEAR REGRESSION MODEL

## TRAIN
prediction_lm_train = lm.predict(x_train)
MSE_lm_train = mean_squared_error(y_train, prediction_lm_train)
R_2_lm_train = r2_score(y_train, prediction_lm_train)
RMSE_lm_train = np.sqrt(MSE_lm_train)


## TEST
prediction_lm_test = lm.predict(x_test)
MSE_lm_test = mean_squared_error(y_test, prediction_lm_test)
R_2_lm_test = r2_score(y_test, prediction_lm_test)
RMSE_lm_test = np.sqrt(MSE_lm_test)



# REGRESSION MODEL USING RANDOM FOREST
## TRAIN
prediction_Rfr_train = Rfr.predict(x_train)
MSE_Rfr_train = mean_squared_error(y_train, prediction_Rfr_train)
R_2_Rfr_train = r2_score(y_train, prediction_Rfr_train)
RMSE_Rfr_train = np.sqrt(MSE_Rfr_train)


## TEST
predictions_Rfr_test = Rfr.predict(x_test)
MSE_Rfr_test = mean_squared_error(y_test, predictions_Rfr_test)
R_2_Rfr_test = r2_score(y_test, predictions_Rfr_test)
RMSE_Rfr_test = np.sqrt(MSE_Rfr_test)


# LASSO REGRESSION MODEL
## TRAIN
prediction_lasso_train = lasso.predict(x_train)
MSE_lasso_train = mean_squared_error(y_train, prediction_lasso_train)
R_2_lasso_train = r2_score(y_train, prediction_lasso_train)
RMSE_lasso_train = np.sqrt(MSE_lasso_train)


## TEST
predictions_lasso_test = lasso.predict(x_test)
MSE_lasso_test = mean_squared_error(y_test, predictions_lasso_test)
R_2_lasso_test = r2_score(y_test, predictions_lasso_test)
RMSE_lasso_test = np.sqrt(MSE_lasso_test)

In [ ]:
# dictionary of results

results_dict = {'Training R-Square Score':
                        {
                            "Linear Regression Model": R_2_lm_train,
                           # "Linear Regression Model p_val": R_2_lm_corr_pval_train,
                            "LASSO": R_2_lasso_train,
                            "Random Forest": R_2_Rfr_train
                        },
                
                    'Test R-Square Score':
                        {
                            "Linear Regression Model": R_2_lm_test,
                            #"Linear Regression Model p_val": R_2_lm_corr_pval_test,
                           "LASSO": R_2_lasso_test,
                            "Random Forest": R_2_Rfr_test
                        },
                
                    'Training RMSE':
                        {
                            "Linear Regression Model": RMSE_lm_train,
                         #   "Linear Regression Model p_val": RMSE_lm_corr_pval_train,
                            "LASSO": RMSE_lasso_train,
                            "Random Forest": RMSE_Rfr_train
                        },
                
                    'Test RMSE':
                        {
                            "Linear Regression Model": RMSE_lm_test,
                          #  "Linear Regression Model p_val": RMSE_lm_corr_pval_test,
                           "LASSO": RMSE_lasso_test,
                            "Random Forest": RMSE_Rfr_test
                        }
                }

# create dataframe from dictionary
results_df = pd.DataFrame(data=results_dict)

pd.options.display.float_format = "{:,.5f}".format
results_df = results_df.astype(float)

### Model Performance Results





In [ ]:
results_df

In [ ]:
# Create figure and axes
f, ax = plt.subplots(figsize=(15,5), nrows=1, ncols=3, sharey=True)

# Create list of titles and predictions to use in for loop
pred = [lm.predict(x_test), Rfr.predict(x_test), lasso.predict(x_test)]
title = ['Linear Regression','Random Forest', 'LASSO Regression']

# Loop through all axes to plot each model's results 
for i in range(3):
    rmse = round(np.sqrt(mean_squared_error(pred[i],y_test)))
    ax[i].set_title(title[i]+"  (RMSE: "+str(rmse)+ ")")
    ax[i].set_xlabel('Actual')
    ax[i].set_ylabel('Predicted')
    ax[i].plot(y_test,y_test,'r')
    ax[i].scatter(y_test,pred[i]) 

In [ ]:
# Create figure and axes
f, ax = plt.subplots(figsize=(5,5))

# Plot on axes
ax.set_title('Linear Regression')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.scatter(y_test,prediction_lm)
ax.plot(y_test,y_test,'r')
plt.show()

In [ ]:
# Create figure and axes
f, ax = plt.subplots(figsize=(5,5))

# Plot on axes
ax.set_title('Random Forest')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.scatter(y_test,predictions_Rfr_test)
ax.plot(y_test,y_test,'r')
plt.show()

In [ ]:
# Create figure and axes
f, ax = plt.subplots(figsize=(5,5))

# Plot on axes
ax.set_title('Lasso Regression')
ax.set_xlabel('Actual')
ax.set_ylabel('Predicted')
ax.scatter(y_test,predictions_lasso_test)
ax.plot(y_test,y_test,'r')
plt.show()

<a id="seven"></a>
## 7. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, we discuss the how the model with the best performance based on the analysis in the previous section works. |

---


From our test results as displayed in the previous section, it can be observed that the Random Forest model gives a better RMSE and R_2 score, thus making it the best model among the three prediction models built.

### Understanding Random Forest Models

A **Random Forest** is a powerful non-parametric algorithm (ie an algorithm that does not make strong assumptions about the form of the mapping function, but instead is free to learn any functional form from the training dataset). It is important to note that non-parametric models like this are good when you have a lot of data with no prior knowledge, and you don’t want to worry too much about choosing just the right features.

Random Forest is an example of an ensemble method built on decision trees. Ensemble methods is a machine learning technique that combines several base models in order to produce one optimal predictive model. In other words, a random forest model relies on aggregating the results of an ensemble of decision trees.